<a href="https://colab.research.google.com/github/hkbu-kennycheng/comp7240/blob/main/lab2_content_based_methods.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 2: content-based methods

We are going to cover another two CF techniques in this lab, baseline method and factorization machine (FM). With another dataset for this lab, it also demonstrate how to import data from csv file in `surprise`.

Let's install `surprise` with `pip` command.

In [ ]:
!pip install surprise

     |████████████████████████████████| 11.8 MB 5.2 MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1619442 sha256=9e033614a99276e3b3d31eba87a12bec996373392c9842f6c77bcee8f6d4a80b
  Stored in directory: /root/.cache/pip/wheels/76/44/74/b498c42be47b2406bd27994e16c5188e337c657025ab400c1c
Successfully built scikit-surprise


# Dataset: CiaoDVD

![](https://url2img-web.herokuapp.com/aHR0cHM6Ly9ndW9ndWliaW5nLmdpdGh1Yi5pby9saWJyZWMvZGF0YXNldHMuaHRtbCNjaWFvZHZk)

In [ ]:
!curl https://guoguibing.github.io/librec/datasets/CiaoDVD.zip > CiaoDVD.zip
!unzip CiaoDVD.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 5678k  100 5678k    0     0  21.6M      0 --:--:-- --:--:-- --:--:-- 21.5M
Archive:  CiaoDVD.zip
  inflating: movie-ratings.txt       
  inflating: readme.txt              
  inflating: review-ratings.txt      
  inflating: trusts.txt              


In [ ]:
!head -n 20 readme.txt

Data Collection Duration: 2013-11 --- 2013-12
Author: Guibing Guo

========================================= Collected Data Formats ==================================
Dataset Name: CiaoDVDs

1. Ciao movie ratings format:  
    1) File: movie-ratings.txt (size: 72,665 --> 72.7K)
    2) Columns: userID, movieID, genreID, reviewID, movieRating, date

2. Ciao review ratings format: 
    1) File: review-ratings.txt (size: 1,625,480 --> 1.6M)
    2) Columns: userID, reviewID, reviewRating
    3) Note: There are users who do not provide movie ratings but provide review ratings.
    
3. Ciao user trusts fromat:
    1) File: trusts.txt (size: 40,133 --> 40K)
    2) Columns: trustorID, trusteeID, trustValue
    3) Note: There are users who may not provide neither movie rating nor review ratings. 



In [ ]:
!head review-ratings.txt

4064,21,3
931,27,4
1869,41,3
44,17,3
9370,26,4
2355,33,3
17616,40,4
17198,29,2
7802,35,4
6567,19,3


## Import data from csv file

In [ ]:
from surprise import Dataset
from surprise import Reader

# path to dataset file
file_path = 'review-ratings.txt'

# seperate user item rating using comma
reader = Reader(sep=',')

data = Dataset.load_from_file(file_path, reader=reader)

## Split training set and testing set

In [ ]:
from surprise.model_selection import train_test_split

# sample random trainset and testset
# test set is made of 25% of the ratings.
trainset, testset = train_test_split(data, test_size=.25)

# Baseline method

In [ ]:
from surprise import BaselineOnly
from surprise import accuracy

algo = BaselineOnly()

# Train the algorithm on the trainset, and predict ratings for the testset
algo.fit(trainset)
predictions = algo.test(testset)

# Then compute RMSE
accuracy.rmse(predictions)

Estimating biases using als...
RMSE: 0.4371


0.43710140568986233

## Evaluation

In [ ]:
from surprise.model_selection import cross_validate

# We can now use this dataset as we please, e.g. calling cross_validate
cross_validate(algo, data, verbose=True)

Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Evaluating RMSE, MAE of algorithm BaselineOnly on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.4334  0.4355  0.4345  0.4333  0.4347  0.4343  0.0008  
MAE (testset)     0.2850  0.2858  0.2853  0.2850  0.2860  0.2854  0.0004  
Fit time          11.07   11.93   11.66   11.94   11.79   11.68   0.32    
Test time         3.13    3.06    3.07    3.05    3.91    3.24    0.33    


{'fit_time': (11.069742679595947,
  11.931014060974121,
  11.65737795829773,
  11.937498092651367,
  11.786023378372192),
 'test_mae': array([0.28504849, 0.28581043, 0.28533574, 0.28500527, 0.2860029 ]),
 'test_rmse': array([0.43340316, 0.43547669, 0.43452349, 0.43328752, 0.43469859]),
 'test_time': (3.1336288452148438,
  3.061807632446289,
  3.067690849304199,
  3.0496153831481934,
  3.9117496013641357)}

# Factorization Machines (FM)

In [ ]:
from surprise import AlgoBase
from surprise import Dataset
from surprise.model_selection import cross_validate
import numpy as np

class MyOwnFM(AlgoBase):

    def __init__(self):

        # Always call base method before doing anything.
        AlgoBase.__init__(self)

    def estimate(self, u, i):
        
        return 3


data = Dataset.load_builtin('ml-100k')
algo = MyOwnFM()

cross_validate(algo, data, verbose=True)